In [226]:
import pandas as pd
import pickle

# 1. Import herbal compounds dataset

In [227]:
df_pca = pd.read_csv("../data/pca/pca_fingerprint_results.csv")
df_pca.head(5)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1


In [228]:
herbal_df_pca = df_pca[df_pca['class'] == 2].reset_index(drop=True)
herbal_df_pca.head(5)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,-0.786607,10.208076,0.203444,0.768367,1.051749,0.249629,0.456858,0.070298,-3.503507,1.049532,...,0.079047,-0.985750,-0.174881,-0.157008,0.159596,-0.726949,0.992478,1.104029,0.457561,2
1,3.082198,14.437355,8.343125,-2.764487,-2.511990,-1.114750,1.334819,-3.459769,0.117543,-2.602732,...,-0.989458,0.396920,-1.331965,0.622666,-0.624093,-0.291460,-0.329094,-0.063016,2.260945,2
2,-1.569839,14.045681,2.108602,-3.467507,-2.492690,-3.873506,-3.066200,2.162735,0.003670,-1.913334,...,-0.269096,-0.779180,-0.044143,-0.097842,0.548767,0.649560,1.069219,0.310761,-0.903324,2
3,-2.530261,10.520718,-2.905435,2.840535,3.649330,3.107620,3.638563,-2.905689,-0.323982,1.863572,...,0.050523,0.262709,0.509218,-0.657400,0.174309,-0.508421,0.334699,-0.355604,-0.057347,2
4,-1.569839,14.045681,2.108602,-3.467507,-2.492690,-3.873506,-3.066200,2.162735,0.003670,-1.913334,...,-0.269096,-0.779180,-0.044143,-0.097842,0.548767,0.649560,1.069219,0.310761,-0.903324,2


In [229]:
herbal_df_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5396 entries, 0 to 5395
Columns: 196 entries, 0 to class
dtypes: float64(195), int64(1)
memory usage: 8.1 MB


# 2. Load SVC Model

In [230]:
optimized_SVC = pickle.load(open("../model/optimized_svc_model.pkl", "rb"))

In [231]:
optimized_SVC.best_estimator_

SVC(C=10, gamma=0.001, probability=True)

# 4. Split Data

In [232]:
X = herbal_df_pca.drop(['class'], axis=1)
y = herbal_df_pca['class']

In [233]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,193,194
0,-0.786607,10.208076,0.203444,0.768367,1.051749,0.249629,0.456858,0.070298,-3.503507,1.049532,...,-0.187665,0.079047,-0.985750,-0.174881,-0.157008,0.159596,-0.726949,0.992478,1.104029,0.457561
1,3.082198,14.437355,8.343125,-2.764487,-2.511990,-1.114750,1.334819,-3.459769,0.117543,-2.602732,...,-2.216753,-0.989458,0.396920,-1.331965,0.622666,-0.624093,-0.291460,-0.329094,-0.063016,2.260945
2,-1.569839,14.045681,2.108602,-3.467507,-2.492690,-3.873506,-3.066200,2.162735,0.003670,-1.913334,...,-0.532587,-0.269096,-0.779180,-0.044143,-0.097842,0.548767,0.649560,1.069219,0.310761,-0.903324
3,-2.530261,10.520718,-2.905435,2.840535,3.649330,3.107620,3.638563,-2.905689,-0.323982,1.863572,...,-0.106645,0.050523,0.262709,0.509218,-0.657400,0.174309,-0.508421,0.334699,-0.355604,-0.057347
4,-1.569839,14.045681,2.108602,-3.467507,-2.492690,-3.873506,-3.066200,2.162735,0.003670,-1.913334,...,-0.532587,-0.269096,-0.779180,-0.044143,-0.097842,0.548767,0.649560,1.069219,0.310761,-0.903324


In [234]:
y.head()

0    2
1    2
2    2
3    2
4    2
Name: class, dtype: int64

In [235]:
y_predict = optimized_SVC.predict(X)
y_predict

array([1, 0, 1, ..., 1, 0, 0])

In [236]:
y_predict_proba = optimized_SVC.predict_proba(X)
y_predict_proba_ligand = y_predict_proba[:,1]
y_predict_proba_ligand

array([0.47044776, 0.27042432, 0.69795019, ..., 0.47605004, 0.16859862,
       0.15588734])

# 5. Concatenate to original data

In [237]:
all_compounds_df = pd.read_csv("../data/results/all_fingerprints.csv")
herbal_df = all_compounds_df[all_compounds_df["class"] == 2][["smiles", "class"]].copy().reset_index(drop=True)
herbal_df.head()

,smiles,class
0,O=C1OC(=O)C2C=CCCC12,2
1,C=CC1SCC=CS1,2
2,OC[C@@H](O)[C@@H](O)[C@H](O)C=O,2
3,CC[C@H](CC[C@@H](C)[C@H]1CC[C@@H]2[C@@]1(CC[C@...,2
4,OC[C@@H](O)[C@H](O)[C@@H](O)C=O,2


In [238]:
herbal_df["prediction"] = pd.Series(y_predict)
herbal_df

,smiles,class,prediction
0,O=C1OC(=O)C2C=CCCC12,2,1
1,C=CC1SCC=CS1,2,0
2,OC[C@@H](O)[C@@H](O)[C@H](O)C=O,2,1
3,CC[C@H](CC[C@@H](C)[C@H]1CC[C@@H]2[C@@]1(CC[C@...,2,1
4,OC[C@@H](O)[C@H](O)[C@@H](O)C=O,2,1
...,...,...,...
5391,CCCCCCC[C@@H](CC(=O)CCC1=CC(=C(C=C1)O)OC)O,2,1
5392,C1[C@H]([C@H](OC2=C1C(=C(C(=C2)O)[C@@H]3[C@H](...,2,0
5393,C1[C@H]([C@H](OC2=C1C(=CC(=C2[C@H]3[C@H]([C@H]...,2,1
5394,C1C(C(OC2=C1C(=CC(=C2C3C(C(OC4=C(C(=CC(=C34)O)...,2,0


In [239]:
herbal_df["probability"] = pd.Series(y_predict_proba_ligand)
herbal_df

,smiles,class,prediction,probability
0,O=C1OC(=O)C2C=CCCC12,2,1,0.470448
1,C=CC1SCC=CS1,2,0,0.270424
2,OC[C@@H](O)[C@@H](O)[C@H](O)C=O,2,1,0.697950
3,CC[C@H](CC[C@@H](C)[C@H]1CC[C@@H]2[C@@]1(CC[C@...,2,1,0.464651
4,OC[C@@H](O)[C@H](O)[C@@H](O)C=O,2,1,0.697950
...,...,...,...,...
5391,CCCCCCC[C@@H](CC(=O)CCC1=CC(=C(C=C1)O)OC)O,2,1,0.821101
5392,C1[C@H]([C@H](OC2=C1C(=C(C(=C2)O)[C@@H]3[C@H](...,2,0,0.175534
5393,C1[C@H]([C@H](OC2=C1C(=CC(=C2[C@H]3[C@H]([C@H]...,2,1,0.476050
5394,C1C(C(OC2=C1C(=CC(=C2C3C(C(OC4=C(C(=CC(=C34)O)...,2,0,0.168599


In [240]:
herbal_df.drop(["class"], axis=1, inplace=True)

# 6. Sort data by probability of similarity to ligand compounds

In [241]:
herbal_df.sort_values(by="probability", ascending=False, inplace=True)
herbal_df

,smiles,prediction,probability
1358,CC(C)([C@H]1[C@@H](C2=C(O1)C(=O)C3=CC=CC=C3C2=...,1,0.995085
4919,CC(C)(C1CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O,1,0.989089
4621,CC1([C@@H](CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O)C,1,0.985031
4635,C/C(=C\CCC(C)(C=C)O)/C(=O)O[C@@H]1CC2=C(C(=O)C...,1,0.982900
1679,CC[C@@]1(O)C(=O)OCC2=C1C=C3N(Cc4cc5ccccc5nc34)...,1,0.981867
...,...,...,...
4049,COc1cc2CCN(C)[C@@H]3Cc4ccccc4c(c1OC)c23,0,0.001198
45,CN1CCC2=CC3=C(C4=C2[C@@H]1CC5=CC(=C(C=C54)OC)O...,0,0.001069
4063,COc1cc2C[C@@H]3N(C)CCc4c(OC)c5OCOc5c(c2cc1OC)c34,0,0.000777
2815,COc1cc2C[C@@H]3N(C)CCc4cc(OC)c(OC)c(c2cc1OC)c34,0,0.000709


In [242]:
predicted_herbal_df = herbal_df[herbal_df["prediction"] == 1]

In [243]:
predicted_herbal_df.head(10)

,smiles,prediction,probability
1358,CC(C)([C@H]1[C@@H](C2=C(O1)C(=O)C3=CC=CC=C3C2=...,1,0.995085
4919,CC(C)(C1CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O,1,0.989089
4621,CC1([C@@H](CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O)C,1,0.985031
4635,C/C(=C\CCC(C)(C=C)O)/C(=O)O[C@@H]1CC2=C(C(=O)C...,1,0.982900
1679,CC[C@@]1(O)C(=O)OCC2=C1C=C3N(Cc4cc5ccccc5nc34)...,1,0.981867
5357,CC1=CC2=C(C(=C1)O)C(=O)C3=C4C(=C5C(=C3C2=O)C(=...,1,0.979280
4632,CC(C)(CC1=C(C2=CC=CC=C2C(=O)C1=O)O)COC(=O)C(C)...,1,0.975674
1779,COC(=O)C1=COC=C2C1=CC=C2C(=O)O,1,0.972391
1350,CC(C)([C@@H]1[C@H](/C(=C(/C2=CC=CC=C2C(=O)OC)\...,1,0.971997
4631,CC(C(C)(CC/C=C(\C)/C(=O)OCC(C)(C)CC1=C(C2=CC=C...,1,0.970818


# 7. Resolve herbal compounds name by dataframe merging

In [244]:
herbal_df_name = pd.read_csv("../data/herbals/herbal_smiles_all.csv")
herbal_df_name

,Senyawa,smiles
0,"2,4-Dihydrophthalic Anhydride",O=C1OC(=O)C2C=CCCC12
1,"2-Vinyl-(4h)-1,3-Dithiin",C=CC1SCC=CS1
2,Arabinose,OC[C@@H](O)[C@@H](O)[C@H](O)C=O
3,b-Sitosterol,CC[C@H](CC[C@@H](C)[C@H]1CC[C@@H]2[C@@]1(CC[C@...
4,D-Xylose,OC[C@@H](O)[C@H](O)[C@@H](O)C=O
...,...,...
5556,[8]-Gingerol,CCCCCCC[C@@H](CC(=O)CCC1=CC(=C(C=C1)O)OC)O
5557,[Epicatechin-(4beta->8)]2-epicatechin-(4beta->...,C1[C@H]([C@H](OC2=C1C(=C(C(=C2)O)[C@@H]3[C@H](...
5558,[Epicatechin-(4beta-8)]2-epicatechin-3-O-gallate,C1[C@H]([C@H](OC2=C1C(=CC(=C2[C@H]3[C@H]([C@H]...
5559,[Gallocatechin-(4alpha->8)]2-catechin,C1C(C(OC2=C1C(=CC(=C2C3C(C(OC4=C(C(=CC(=C34)O)...


In [245]:
final_predicted_herbal_df = pd.merge(predicted_herbal_df, herbal_df_name, on="smiles", how="inner")
final_predicted_herbal_df

,smiles,prediction,probability,Senyawa
0,CC(C)([C@H]1[C@@H](C2=C(O1)C(=O)C3=CC=CC=C3C2=...,1,0.995085,Avicequinone A
1,CC(C)(C1CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O,1,0.989089,Stenocarpoquinone B
2,CC1([C@@H](CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O)C,1,0.985031,Rhinacanthin A
3,C/C(=C\CCC(C)(C=C)O)/C(=O)O[C@@H]1CC2=C(C(=O)C...,1,0.982900,Rhinacanthin P
4,CC[C@@]1(O)C(=O)OCC2=C1C=C3N(Cc4cc5ccccc5nc34)...,1,0.981867,Camptothecin
...,...,...,...,...
3055,OC[C@H]1O[C@H]([C@H](O)[C@@H](O)[C@@H]1O)c2c(O...,1,0.416244,Mangiferin
3056,OC[C@H]1O[C@H]([C@H](O)[C@@H](O)[C@@H]1O)c2c(O...,1,0.416244,"6-C-beta-D-Glucopyranosyl-5,7,2',4',5'-pentahy..."
3057,CC(=O)O[C@H]1C[C@H](C([C@]2([C@]1([C@H]3CC4=C(...,1,0.416133,Caesalpinin C
3058,C[C@@H]1[C@@H]2CC[C@@]3([C@@]([C@H]2CC4=C1C=CO...,1,0.416133,Caesalpinin MA


In [246]:
final_predicted_herbal_df.rename(columns={"Senyawa": "compound_name","smiles":"compound_smiles"}, inplace=True)
final_predicted_herbal_df.insert(0, "compound_name", final_predicted_herbal_df.pop("compound_name"))
final_predicted_herbal_df

,compound_name,compound_smiles,prediction,probability
0,Avicequinone A,CC(C)([C@H]1[C@@H](C2=C(O1)C(=O)C3=CC=CC=C3C2=...,1,0.995085
1,Stenocarpoquinone B,CC(C)(C1CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O,1,0.989089
2,Rhinacanthin A,CC1([C@@H](CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O)C,1,0.985031
3,Rhinacanthin P,C/C(=C\CCC(C)(C=C)O)/C(=O)O[C@@H]1CC2=C(C(=O)C...,1,0.982900
4,Camptothecin,CC[C@@]1(O)C(=O)OCC2=C1C=C3N(Cc4cc5ccccc5nc34)...,1,0.981867
...,...,...,...,...
3055,Mangiferin,OC[C@H]1O[C@H]([C@H](O)[C@@H](O)[C@@H]1O)c2c(O...,1,0.416244
3056,"6-C-beta-D-Glucopyranosyl-5,7,2',4',5'-pentahy...",OC[C@H]1O[C@H]([C@H](O)[C@@H](O)[C@@H]1O)c2c(O...,1,0.416244
3057,Caesalpinin C,CC(=O)O[C@H]1C[C@H](C([C@]2([C@]1([C@H]3CC4=C(...,1,0.416133
3058,Caesalpinin MA,C[C@@H]1[C@@H]2CC[C@@]3([C@@]([C@H]2CC4=C1C=CO...,1,0.416133


In [248]:
final_predicted_herbal_df.to_csv("../data/results/herbal_prediction.csv", index=False)

# FINAL RESULT

Now, we got the herbal compounds with the highest probability toward ligand compound. So, these compounds can be considered as candidate compound as the remedy of Alzheimer Diseases. These top 10 compounds are:

In [249]:
final_predicted_herbal_df.head(10)

,compound_name,compound_smiles,prediction,probability
0,Avicequinone A,CC(C)([C@H]1[C@@H](C2=C(O1)C(=O)C3=CC=CC=C3C2=...,1,0.995085
1,Stenocarpoquinone B,CC(C)(C1CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O,1,0.989089
2,Rhinacanthin A,CC1([C@@H](CC2=C(O1)C(=O)C3=CC=CC=C3C2=O)O)C,1,0.985031
3,Rhinacanthin P,C/C(=C\CCC(C)(C=C)O)/C(=O)O[C@@H]1CC2=C(C(=O)C...,1,0.982900
4,Camptothecin,CC[C@@]1(O)C(=O)OCC2=C1C=C3N(Cc4cc5ccccc5nc34)...,1,0.981867
5,Xylospyrin,CC1=CC2=C(C(=C1)O)C(=O)C3=C4C(=C5C(=C3C2=O)C(=...,1,0.979280
6,Rhinacanthin L,CC(C)(CC1=C(C2=CC=CC=C2C(=O)C1=O)O)COC(=O)C(C)...,1,0.975674
7,Cerberic acid,COC(=O)C1=COC=C2C1=CC=C2C(=O)O,1,0.972391
8,Avicennone B,CC(C)([C@@H]1[C@H](/C(=C(/C2=CC=CC=C2C(=O)OC)\...,1,0.971997
9,Rhinacanthin K,CC(C(C)(CC/C=C(\C)/C(=O)OCC(C)(C)CC1=C(C2=CC=C...,1,0.970818
